### Multi-camera recording to CSV 128px

In [2]:
import cv2
import numpy as np
import pandas as pd
import os
import glob

C:\Users\senne\AppData\Local\Temp\ipykernel_19152\4074716408.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [16]:
recording_dir = '024/'
amount_of_pixels = '40px' # _AMOUNT OF PIXELS_
sequence_number = '024' # _SEQUENCE NUMBER_, a number that is unique to the recording sequence

In [17]:
mp4_files = glob.glob(recording_dir + '*.mp4')

if not mp4_files:
    raise ValueError('No mp4 files found in the recording directory')

df = pd.DataFrame()

lower_red = np.array([0, 0, 100], dtype=np.uint8)
upper_red = np.array([75, 75, 255], dtype=np.uint8)

i=0

for video in mp4_files:
        
    cap = cv2.VideoCapture(video)
    mean_x = []
    mean_y = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        # create a mask for the red pixels in the frame
        mask = cv2.inRange(frame, lower_red, upper_red)
        # apply the mask to the frame so that only the red pixels are shown
        detected_ball = cv2.bitwise_and(frame, frame, mask=mask)    
        # get all the coordinates of red pixels in the frame
        red_coordinates = np.column_stack(np.where((frame[:, :, 0] >= lower_red[0]) & (frame[:, :, 0] <= upper_red[0]) &
        (frame[:, :, 1] >= lower_red[1]) & (frame[:, :, 1] <= upper_red[1]) &
        (frame[:, :, 2] >= lower_red[2]) & (frame[:, :, 2] <= upper_red[2])
        ))    
        # flip the y coordinates to match the origin at the bottom left
        red_coordinates[:, 0] = frame.shape[0] - red_coordinates[:, 0]
        if red_coordinates.size == 0:
            # add NaN if no red ball is detected
            mean_x.append('NaN')
            mean_y.append('NaN')
        else:
            mean_x.append(np.mean(red_coordinates[:, 1]))
            mean_y.append(np.mean(red_coordinates[:, 0]))
    
    # add the median x and y coordinates to the dataframe
    df[f'u{i}'] = mean_x
    df[f'v{i}'] = mean_y
    cap.release()
    
    i+=1
# Make sure the 'data' directory exists
if not os.path.exists('data'):
    os.makedirs('data')

# Save the dataframe to a CSV file
output_file_name_of_red_ball_coordinates = (
        'red_ball_coordinates_' + amount_of_pixels + '_' +  sequence_number +'.csv')
df.to_csv(f'data/' + output_file_name_of_red_ball_coordinates , index=False)

red_ball_coordinates_40px_024.csv
            u0         v0         u1         v1        u2         v2  \
0          8.5       17.5       35.0      18.25       7.0       32.0   
1          8.5  18.666667       34.5       19.0       6.5       34.0   
2          8.5       19.5  34.666667  19.333333       7.4       33.6   
3          8.5       20.0       34.4       20.8       7.4       34.0   
4          8.5      20.75       34.5       21.5       NaN        NaN   
...        ...        ...        ...        ...       ...        ...   
1018       4.5   4.222222  36.642857   2.357143       3.6       19.4   
1019       4.5   4.222222  36.692308   2.076923       3.6       19.4   
1020      4.68        4.6  36.583333   3.666667  3.857143  20.428571   
1021   4.52381   5.285714  37.090909   4.272727  3.857143  20.428571   
1022  4.846154   6.192308  36.526316   4.947368       3.0     20.875   

             u3         v3         u4         v4        u5         v5   u6  \
0          21.5       3